# Krakenbot usage example

Run this cell in case of module import problems.

In [2]:
import sys
import os
path = os.getenv('KRAKENBOTPATH')
sys.path.append(path)
import krakenbot

None


ModuleNotFoundError: No module named 'krakenbot'

Import required modules.

In [1]:
from krakenbot.database.connectors.sqlite import SQLiteConnector
from krakenbot.rest.bot import KrakenBot

ModuleNotFoundError: No module named 'krakenbot'

Create the database connector and `KrakenBot` instances.

In [3]:
# Input your database name.
db_name = 'test.db'
connector = SQLiteConnector(db_name=db_name)
bot = KrakenBot(db_connector=connector)


Fetches the latest batch of transactions (up to 1000).

In [4]:
# Change the batch size (up to 1000 positions) and the tradable asset pair
size = 3
pair = 'XBTUSD'
recent_trades = bot.recent_trades(size=size, pair=pair)

Prepare column names.

In [5]:
trades = recent_trades[pair]
columns = ['"pair name"', 'price', 'volume', 'time', '"buy(b)/sell(s)"', '"market(m)/limit(l)"', 'miscellaneous']
columns_names = " ,".join(columns)

Create the example table.

In [6]:
# Rename the table if you want.
table_name = 'recent_trades'
query_table = f'create table if not exists {table_name}  ("trade_id" integer primary key autoincrement, {columns_names})'
connector.execute(query_table)

Fill the table with the recent trade data.

In [7]:
# Change the batch size (up to 1000 positions) and the tradable asset pair
size = 3
pair = 'XBTUSD'
bot.update_recent_trades(table_name=table_name, pair=pair, size=size)


Check the data inside the table.

In [8]:
connector.execute(f'select * from {table_name}')

[(0, 'XBTUSD', '20025.50000', '0.00050049', 1663248661.2781415, 'b', 'l', ''),
 (1, 'XBTUSD', '20025.50000', '0.00675000', 1663248661.885247, 'b', 'l', ''),
 (2, 'XBTUSD', '20023.00000', '0.00011206', 1663248665.6921606, 'b', 'l', '')]

Update the table with the newest data.

In [9]:
bot.update_recent_trades(table_name=table_name, pair=pair, size=size)
connector.execute(f'select * from {table_name}')

[(0, 'XBTUSD', '20023.00000', '0.00011206', 1663248665.6921606, 'b', 'l', ''),
 (1, 'XBTUSD', '20023.70000', '0.00194559', 1663248667.3217237, 'b', 'm', ''),
 (2, 'XBTUSD', '20022.40000', '0.01989883', 1663248672.5856986, 'b', 'm', '')]

Repeat this action every 20 seconds for a minute and save the log info into the log file.

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.DEBUG)
bot.repeat_action(action_name='update_recent_trades', duration=60, interval=20, table_name=table_name, size=size, pair=pair)

16:19:21,652 urllib3.connectionpool DEBUG Starting new HTTPS connection (1): api.kraken.com:443
16:19:22,56 urllib3.connectionpool DEBUG https://api.kraken.com:443 "GET /0/public/Trades?pair=XBTUSD HTTP/1.1" 200 None
16:19:22,105 krakenbot.rest.bot INFO Updated row: [(0, 'XBTUSD', '20028.40000', '0.00062010', 1663248675.3773704, 'b', 'l', '')]
16:19:22,116 krakenbot.rest.bot INFO Updated row: [(1, 'XBTUSD', '20029.70000', '0.00552315', 1663248675.3783123, 'b', 'l', '')]
16:19:22,128 krakenbot.rest.bot INFO Updated row: [(2, 'XBTUSD', '20029.60000', '0.01300400', 1663248677.563743, 's', 'm', '')]
16:19:22,129 krakenbot.rest.bot INFO The action update_recent_trades executed on date 2022-09-15 16:19:22.129356
16:19:23,970 krakenbot.rest.bot INFO The action update_recent_trades has been repeated every 20.0 seconds for overall 60.0 seconds.
16:19:42,140 urllib3.connectionpool DEBUG Starting new HTTPS connection (1): api.kraken.com:443
16:19:42,841 urllib3.connectionpool DEBUG https://api.kr

Check our table changes.

In [11]:
select_recent_trades = connector.execute(f'select * from {table_name}')
select_recent_trades

[(0, 'XBTUSD', '20023.00000', '0.00011206', 1663248665.6921606, 'b', 'l', ''),
 (1, 'XBTUSD', '20023.70000', '0.00194559', 1663248667.3217237, 'b', 'm', ''),
 (2, 'XBTUSD', '20022.40000', '0.01989883', 1663248672.5856986, 'b', 'm', '')]

16:18:43,58 urllib3.connectionpool DEBUG Starting new HTTPS connection (1): api.kraken.com:443
16:18:43,497 urllib3.connectionpool DEBUG https://api.kraken.com:443 "GET /0/public/Trades?pair=XBTUSD HTTP/1.1" 200 None
16:18:43,551 krakenbot.rest.bot INFO Updated row: [(0, 'XBTUSD', '20022.50000', '0.12468381', 1663248672.5867398, 'b', 'm', '')]
16:18:43,564 krakenbot.rest.bot INFO Updated row: [(1, 'XBTUSD', '20022.30000', '0.00424243', 1663248672.6808083, 'b', 'l', '')]
16:18:43,575 krakenbot.rest.bot INFO Updated row: [(2, 'XBTUSD', '20023.70000', '0.01838757', 1663248672.681862, 'b', 'l', '')]
16:18:43,576 krakenbot.rest.bot INFO The action update_recent_trades executed on date 2022-09-15 16:18:43.576068
16:19:03,586 urllib3.connectionpool DEBUG Starting new HTTPS connection (1): api.kraken.com:443
16:19:03,904 urllib3.connectionpool DEBUG https://api.kraken.com:443 "GET /0/public/Trades?pair=XBTUSD HTTP/1.1" 200 None
16:19:03,945 krakenbot.rest.bot INFO Updated row: [(0, 'XBTUSD', '